In [1]:
# 复现RESNET 18（普通）以及RESNET 50 （有bottleneck）
# RESNET 18：https://towardsdev.com/implement-resnet-with-pytorch-a9fb40a77448
# RESNET 50：https://github.com/liao2000/ML-Notebook/blob/main/ResNet/ResNet_PyTorch.ipynb
# 训练：15.【代码】ResNet代码详解（下载见附件）.pdf

In [1]:
import torch
import torch.nn as nn
from torchsummary import summary

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class ResBlock(nn.Module):
    def __init__(self, down_sample, in_channels, out_channels):
        super().__init__()
        # 是否需要下采样
        self.down_sample = down_sample
        if down_sample:
            # 需要下采样
            self.conv1 = nn.Conv2d(in_channels, out_channels, 3, 2, 1)
            # 短路连接
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, 2, 0),
                nn.BatchNorm2d(out_channels),
            )
            
            
        else:
            # 不需要下采样
            self.conv1 = nn.Conv2d(in_channels, out_channels, 3, 1, 1)
            # 短路连接
            self.shortcut = nn.Sequential()


        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU()

        # conv2
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()

        self.relu3 = nn.ReLU()


    def forward(self, x):
        shortcut = self.shortcut(x)

        # conv1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)

        # conv2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        # shortcut
        x = x + shortcut

        x = self.relu3(x)

        return x



In [3]:
resblock = ResBlock(True, 128, 256)

In [4]:
resblock.to(device)

ResBlock(
  (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (shortcut): Sequential(
    (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (relu3): ReLU()
)

In [5]:
summary(resblock, (128, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 256, 14, 14]          33,024
       BatchNorm2d-2          [-1, 256, 14, 14]             512
            Conv2d-3          [-1, 256, 14, 14]         295,168
       BatchNorm2d-4          [-1, 256, 14, 14]             512
              ReLU-5          [-1, 256, 14, 14]               0
            Conv2d-6          [-1, 256, 14, 14]         590,080
       BatchNorm2d-7          [-1, 256, 14, 14]             512
              ReLU-8          [-1, 256, 14, 14]               0
              ReLU-9          [-1, 256, 14, 14]               0
Total params: 919,808
Trainable params: 919,808
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.38
Forward/backward pass size (MB): 3.45
Params size (MB): 3.51
Estimated Total Size (MB): 7.34
-------------------------------------------

In [6]:
class Resnet18(nn.Module):
    """
    搭建简单的残差网络：resnet18
    input：244 * 244 * 3 RGB彩色图
    output： 1000类
    """

    def __init__(self, num_classes):
        super().__init__()

        # layer 0
        self.layer0 = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 1)
        )

        # 每层通道数量
        channels_list = [64, 128, 256, 512]

        # layer 1
        self.layer1 = nn.Sequential(
            # 不做下采样
            ResBlock(False, channels_list[0], channels_list[0]),
            # 不做下采样
            ResBlock(False, channels_list[0], channels_list[0]),
        )


        # layer 2
        self.layer2 = nn.Sequential(
            # 做下采样
            ResBlock(True, channels_list[0], channels_list[1]),
            # 不做下采样
            ResBlock(False, channels_list[1], channels_list[1]),
        )


        # layer 3
        self.layer3 = nn.Sequential(
            # 不做下采样
            ResBlock(True, channels_list[1], channels_list[2]),
            # 不做下采样
            ResBlock(False, channels_list[2], channels_list[2]),
        )


        # layer 4
        self.layer4 = nn.Sequential(
            # 做下采样
            ResBlock(True, channels_list[2], channels_list[3]),
            # 不做下采样
            ResBlock(False, channels_list[3], channels_list[3]),
        )


        # layer 5
        
        self.layer5 = nn.Sequential(
            # AAP
            nn.AdaptiveAvgPool2d((1, 1)),
            # flatten
            nn.Flatten(start_dim=1),
            # FC
            nn.Linear(channels_list[3], num_classes)
        )



    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)

        return x

In [7]:
res18 = Resnet18(10).to(device)

In [8]:
summary(res18, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,928
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,928
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
             ReLU-11           [-1, 64, 56, 56]               0
         ResBlock-12           [-1, 64, 56, 56]               0
           Conv2d-13           [-1, 64, 56, 56]          36,928
      BatchNorm2d-14           [-1, 64,

In [9]:
import torch
import torchvision
import torchvision.transforms as transforms

In [10]:
import torch.utils


transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [11]:
optimizer = torch.optim.SGD(res18.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [12]:
for epoch in range(2):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        res18.zero_grad()
        # # zero the parameter gradients
        # optimizer.zero_grad()

        # forward + backward + optimize
        outputs = res18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # print statistics
        running_loss += loss.item()
        # print(loss.item())
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0


        # break

    # break

[1,  2000] loss: 2.681
[1,  4000] loss: 2.279
[1,  6000] loss: 2.152
[1,  8000] loss: 2.002
[1, 10000] loss: 1.907
[1, 12000] loss: 1.911


KeyboardInterrupt: 